# Homework 1

#### EE-556 Mathematics of Data - Fall 2022

If you worked with somebody else on this Homework, fill in their names here:

For the first notebook about theory, I exchanged ideas with Arvind Menon Satish.

# Logistic Regression - 10 Points


We consider a binary classification task that we will model using logistic regression. Your goal will be to find a classifier using first-order methods and accelerated gradient descent methods. The first part will consist of more theoretical questions, and the second and third part will ask you to implement these methods.



Logistic regression is a classic approach to _binary classification_. Before we dive in, let us first define the standard logistic function $\sigma$ on which most of what follows is built:

\begin{equation*}
    \sigma : x \mapsto \frac{1}{1 + \exp{(-x)}}.
\end{equation*}

In logistic regression, we model the _conditional probability_ of observing a class label $b$ given a set of features $\mathbf{a}$. More formally, if we observe $n$ independent samples

\begin{equation*}
    \{(\mathbf{a}_i,b_i)\}_{i=1}^n,
\end{equation*}

where $\mathbf{a}_i\in\mathbb{R}^p$ and $b_i\in\{0, 1\}$ is the class label, we _assume_ that $b_i$ given $\mathbf{a}_i$ is a Bernouilli random variable with parameter $\sigma(\mathbf{a}_i^T\mathbf{x}^\natural)$, for some unknown $\mathbf{x}^\natural \in \mathbb{R}^p$. In other words, we assume that there exists an $\mathbf{x}^\natural \in \mathbb{R}^p$ such that

\begin{equation*}
    \mathbb{P}(b_i = 1 \mid \mathbf{a}_i) = \sigma(\mathbf{a}_i^T\mathbf{x}^\natural) \quad \text{ and } \quad \mathbb{P}(b_i = 0 \mid \mathbf{a}_i) = 1 - \sigma(\mathbf{a}_i^T\mathbf{x}^\natural)=  \sigma( - \mathbf{a}_i^T\mathbf{x}^\natural).
\end{equation*}

This is our statistical model. It can be written in a more compact form as follows,

\begin{equation*}
    \mathbb{P}(b_i = j \mid \mathbf{a}_i) = \sigma(\mathbf{a}_i^T\mathbf{x}^\natural)^j\sigma(-\mathbf{a}_i^T\mathbf{x}^\natural)^{1-j}, \quad j \in \{0, 1\}.
\end{equation*}

Our goal now is to determine the unknown $\mathbf{x}^\natural$ by constructing an estimator.

We are provided with a set of $n$ independent observations, we can write down the negative log-likelihood $f$ as follows:

\begin{equation*}
	\begin{aligned}
		f(\mathbf{x}) = -\log(\mathbb{P}(b_1, \dots, b_n | a_1, \dots, a_n)) & = -\log \left( \prod_{i=1}^{n} \mathbb{P}(b_i|a_i) \right) \quad \quad \text{(by independence)}\\
		 & =  \sum_{i=1}^n - \log(\sigma(\mathbf{a}_i^T\mathbf{x}^\natural)^{b_i}\sigma(-\mathbf{a}_i^T\mathbf{x}^\natural)^{1-b_i}) \\
		 & = \sum_{i=1}^n  b_i \log(1 + \exp(- \mathbf{a}_i^T\mathbf{x})) + (1-b_i)\log(1 + \exp(\mathbf{a}_i^T\mathbf{x})).
	\end{aligned}
\end{equation*}


__(a)__ (1 point) Show that the function $u \mapsto \log(1 + \exp(u))$ is convex. Deduce that $f$ is convex.

Let $g(u) = \log(1+e^u)$. Then, in order to show that $g(u)$ is convex, we show that $\forall u \in \mathbb{R}: \frac{d^2g}{du^2} \geq 0$.

\begin{equation*}
    \begin{aligned}
    \frac{dg}{du} = & \frac{e^u}{1 + e^u}\\
    \frac{d^2g}{du^2} = &\frac{e^u}{(1 + e^u)^2}
    \end{aligned}
\end{equation*}

Since $\forall u \in \mathbb{R}: e^u \geq 0$, we know that $\forall u \in \mathbb{R}: \frac{d^2g}{du^2} \geq 0$, and it's domain is $\mathbb{R}$. Hence, $g(u)$ is a convex function. Furthermore, we know that since $g(u)$ is convex, $\forall c \geq 0: c \cdot g(u)$ is convex. We also know that for any convex functions $g(u)$ and $h(u)$, $g(u) + h(u)$ is still convex.

$f(x) = \sum_{i=1}^n  b_i \log(1 + \exp(- \mathbf{a}_i^T\mathbf{x})) + (1-b_i)\log(1 + \exp(\mathbf{a}_i^T\mathbf{x})) = \sum_{i=1}^n  b_i g(\mathbf{a}_i^T\mathbf{x}) + (1-b_i)g(\mathbf{a}_i^T\mathbf{x})$.

From our previous two statements, and the fact that $b_i \geq 0$ and $1 - b_i \geq 0$, we deduce that $b_i g(\mathbf{a}_i^T\mathbf{x}) + (1-b_i)g(\mathbf{a}_i^T\mathbf{x})$ is convex. Hence, $f(x) = \sum_{i=1}^n b_i g(\mathbf{a}_i^T\mathbf{x}) + (1-b_i)g(\mathbf{a}_i^T\mathbf{x})$ must also be convex, as it is a sum of convex functions.

You have just established that the negative log-likelihood is a convex function. So in principle, any local minimum of the maximum likelihood estimator, which is defined as

\begin{equation*}
    \mathbf{x}^\star_{ML} = \arg\min_{\mathbf{x} \in \mathbb{R}^p} f(\mathbf{x}),
\end{equation*}

is a global minimum so it can serve as an estimator of $\mathbf{x}^\natural$. But, does the minimum always exist? We will ponder this question in the following three points.

__(b)__ (1 point) Explain the difference between infima and minima.  Give an example of a convex function, defined over $\mathbb{R}$, that does not attain its infimum. 

The infimum of a set $S$ is the largest lower bound on $S$. Imagine the set $\mathbb{N \cup \{0\}}$. Here, $0$ is both the minimum value as well as the greatest lower bound on the set. However, if we consider the set $\mathbb{R}^+ \setminus \{0\}$, there exists no minumum, since $(\forall x \in \mathbb{R}^+ \setminus \{0\})(\exists y \in \mathbb{R}^+ \setminus \{0\}): y < x$. Since there is no such value in the set, the infimum considers the largest lower bound on this set, which would be $0$.

Let $f(x) = \log(1 + e^x)$. For this function, $Range(f) = (0, \infty]$. Hence, the infimum of $f(x) = 0$, but the minimum does not exists over the domain $\mathbb{R}$.

__(c)__ (1 points) Let us assume that there exists $\mathbf{x}_0 \in \mathbb{R}^p$ such that 

\begin{equation*}
    \forall i\in\{1, \dots, n\}, \quad \quad \text{ if } b_i = 1, \text{ then } \mathbf{a}_i^T\mathbf{x}_0 > 0, \text{ and if } b_i = 0 \text{ then } \mathbf{a}_i^T\mathbf{x}_0 < 0.
\end{equation*}

This is called _complete separation_ in the literature. Can you think of a geometric reason why this name is appropriate? Think of a 2D example where this can happen (i.e $p=2$) and describe why _complete separation_ is an appropriate name.



If we are in for example $\mathbb{R^2}$, then $a_i^T x_0 = 0$ would represent a line of possible values for which this holds. Then, $a_i^T \cdot x_0 > 0$ and $a_i^T \cdot x_0 < 0$ are both on "opposite" sides of the line, completely separating $\mathbb{R}^2$ into two segments, each representing a different value of $b_i$. This idea would generalize to hyperplanes in hyperspaces, which would be separated into two parts that are complete together (representing $b_i = 0$ and $b_i = 1$. For this reason, the name could be seen as appropriate. 

From this, you should see that it is likely that our data satisfies the complete separation assumption. Unfortunately, as you will show in the following question, this can become an obstacle.

__(d)__ (2 points) In a _complete separation_ setting, i.e, there exists $\mathbf{x}_0$ such that 

\begin{equation*}
    \forall i\in\{1, \dots, n\}, \quad \quad \text{ if } b_i = 1, \text{ then } \mathbf{a}_i^T\mathbf{x}_0 > 0, \text{ and if } b_i = 0 \text{ then } \mathbf{a}_i^T\mathbf{x}_0 < 0.
\end{equation*}

prove that the function $f$ does not attain its minimum. 

__Hint__: If the function did have a minimum, would it be above, below or equal to zero? Then think of how $f(2 \mathbf{x}_0)$ compares with $f(\mathbf{x}_0)$, how about $f(\alpha \mathbf{x}_0)$ for $\alpha \rightarrow + \infty$ ? 

Let's remind ourselves that $f(x_0) = \sum_{i=1}^n  b_i \log(1 + \exp(- \mathbf{a}_i^T\mathbf{x_0})) + (1-b_i)\log(1 + \exp(\mathbf{a}_i^T\mathbf{x_0}))$.

We can split the function into two components, $f(x_0) = \sum_{i|b_i = 1}^n  \log(1 + \exp(- \mathbf{a}_i^T\mathbf{x_0})) + \sum_{i|b_i = 0} \log(1 + \exp(\mathbf{a}_i^T\mathbf{x_0}))$. We can rewrite the function as $f(x_0) = \sum_{i|b_i = 1}^n  \log(1 + \exp(- \mathbf{a}_i^T\mathbf{x_0})) + \sum_{i|b_i = 0} \log(1 + \exp(\mathbf{a}_i^T\mathbf{x_0}))$. However, from the complete separation property, we notice that in the first sum, $\log(1 + \exp(-\mathbf{a}_i^T\mathbf{x_0}))$ and $-\mathbf{a}_i^T\mathbf{x_0} < 0$ for all terms in the sum, the sum would never reach its true value of $0$, since the function in the sum can never reach $0$. The same argument holds for the second sum, since $\log(1 + \exp(\mathbf{a}_i^T\mathbf{x_0}))$ and $\mathbf{a}_i^T\mathbf{x_0} < 0$ for all elements in the sum. If we now assume that $f(x_0)$ is minimum, we can construct a new argument $x = 2 \cdot x_0$, which would make $f(x)$ even lower than $f(x_0)$, since $a_i^T \cdot x < a_i^T \cdot x_0$. This is a contradiction to the assumption that $x_0$ is minimum, hence the function does not have a minimum value. It reaches from above towards $0$, which would be the minimum (and is the infinum).

As you have just shown convex functions do not always attain their infimum. So it is possible for the maximum-likelihood estimator $\mathbf{x}^\star_{ML}$ to not exist. We will resolve this issue by adding a regularizer. 

In what follows, we consider the function

\begin{equation*}
	f_\mu(\mathbf{x}) = f(\mathbf{x}) + \frac{\mu}{2}\|\mathbf{x}\|_2^2
\end{equation*}
with $\mu> 0$.

__(e)__ (1 points) Show that the gradient of $f_\mu$ can be expressed as 
\begin{equation}
	\nabla f_\mu(\mathbf{x}) = \sum_{i=1}^n (\sigma(\mathbf{a}_i^T\mathbf{x}) - b_i)\mathbf{a}_i + \mu \mathbf{x}.
\tag{1}\label{gradient}
\end{equation}
__Hint__: Lecture 3 shows you how to proceed with this question.

Let $g(x) = \frac{\mu}{2} \Vert x \Vert_2^2$. Then $J_{f_\mu}(x) = J_f(x) + J_g(x)$.

We first compute $J_f(x)$ and $J_g(x)$ below.

\begin{equation*}
    \begin{aligned}
    J_f(x) &= \sum_{i=1}^n \left( - \frac{b_ie^{-a_i^Tx}}{1 + e^{-a_i^Tx}} + \frac{(1-b_i)_ie^{a_i^Tx}}{1 + e^{a_i^Tx}}\right) a_i^T\\
    &= \sum_{i=1}^n \left( - \frac{b_i}{1 + e^{a_i^Tx}} + \frac{(1-b_i)}{1 + e^{-a_i^Tx}}\right) a_i^T\\
    &= \sum_{i=1}^n \left(\sigma(a_i^Tx) - b_i (\frac{1}{1 + e^{a_i^Tx}} + \frac{1}{1 + e^{-a_i^Tx}})\right) a_i^T\\
    &= \sum_{i=1}^n \left(\sigma(a_i^Tx) - b_i\right) a_i^T\\
    J_g(x) & = \mu x^T\\
    \end{aligned}
\end{equation*}

The gradient of $f_\mu(x)$ can be obtained by taking the tranpose of its Jacobian.

\begin{equation*}
    \begin{aligned}
    \nabla f_\mu(x) &= (J_f(x) + J_g(x))^T\\
    \nabla f_\mu(x) &= \sum_{i=1}^n \left(\sigma(a_i^Tx) - b_i\right) a_i + \mu x
    \end{aligned}
\end{equation*}

__(f)__ (1 points) Show that the Hessian of $f_\mu$ can be expressed as 
\begin{equation}
	\nabla^2 f_\mu(\mathbf{x}) = \sum_{i=1}^{n} \sigma(\mathbf{a}_i^T\mathbf{x})\sigma(- \mathbf{a}_i^T\mathbf{x})\mathbf{a}_i\mathbf{a}_i^T + \mu \mathbf{I}.
\tag{2}\label{eq:hessian}
\end{equation}




In a similar manner as before, we want to compute $J_{\nabla f_\mu(x)}(x)$.

\begin{equation*}
    \begin{aligned}
    J_{\nabla f_\mu}(x) &= \sum_{i=1}^n \frac{e^{-a_i^Tx}}{(1 + e^{-a_i^Tx})^2}a_ia_i^T + \mu I\\
    J_{\nabla f_\mu}(x) &= \sum_{i=1}^n \frac{1}{1 + e^{-a_i^Tx}}\frac{e^{-a_i^Tx}e^{a_i^Tx}}{(1 + e^{-a_i^Tx})e^{a_i^Tx}}a_ia_i^T + \mu I\\
    J_{\nabla f_\mu}(x) &= \sum_{i=1}^n \frac{1}{1 + e^{-a_i^Tx}}\frac{1}{1 + e^{a_i^Tx}}a_ia_i^T + \mu I\\
    J_{\nabla f_\mu}(x) &= \sum_{i=1}^n \sigma(a_i^Tx)\sigma(-a_i^Tx)a_ia_i^T + \mu I\\
    \end{aligned}
\end{equation*}

Then, $\nabla^2 f_\mu(x) = J_{\nabla f_\mu}(x)^T = \sum_{i=1}^n \sigma(a_i^Tx)\sigma(-a_i^Tx)a_ia_i^T + \mu I$ (Cool! The transpose of the Jacobian of the gradient is the Hessian; it's symmetric :D)

It is convenient to observe that we can write the Hessian in a more compact form by defining the matrix
\begin{equation}
	\mathbf{A} = \begin{bmatrix}
        \leftarrow &  \mathbf{a}_1^T & \rightarrow \\
        \leftarrow &  \mathbf{a}_2^T & \rightarrow \\
         &  \ldots &  \\
        \leftarrow &  \mathbf{a}_n^T & \rightarrow \\
  \end{bmatrix}.
\end{equation}
It is easy to see that we have
\begin{equation}
	\nabla^2 f_\mu(\mathbf{x}) =  \mathbf{A}^T \text{Diag}\left( \sigma(\mathbf{a}_i^T\mathbf{x})\sigma(- \mathbf{a}_i^T\mathbf{x}) \right)\mathbf{A}+ \mu \mathbf{I}.
\end{equation}


__(g)__ (1 point) Show that $f_\mu$ is $\mu$-strongly convex. 

In order to show that $f_\mu(x)$ is $\mu$-strongly convex, we will show that $\nabla^2 f_\mu(x) \succcurlyeq \mu I \Rightarrow \nabla^2 f(x) + \mu I \succcurlyeq \mu I \Rightarrow \nabla^2 f(x) \succcurlyeq 0$. So, we must show $\nabla^2 f(x) = A^T \text{Diag}\left(\sigma(a_i^Tx)\sigma(-a_i^Tx)\right)A \succcurlyeq 0$. However, since we showed $f(x)$ is convex in **(a)**, it is a property that $\nabla^2 f(x) \succcurlyeq 0$. Thus, $f_\mu(x)$ is $\mu$-strongly convex.

__(h)__ (1 point) Is it possible for a strongly convex function, defined over $\mathbb{R}^p$, to not attain its minimum ? <a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1) Justify your reasoning.

No, this is not possible. Let $g(x)$ be a strongly convex function. Then, we know by strong convexity that $\forall x, y \in D_g: f(y) \geq f(x) + \langle \nabla f(x), y - x \rangle + \frac{\mu}{2} \Vert y - x \Vert_2^2$, where $D_g$ is the domain of $g(x)$. From what we have seen earlier, a continuous function that does not attain its minimum would approach it when it's argument approaches $\pm \infty$. However, by the property above, $\forall y \in D_g$, it must lay above a polynomial globally. This means that it cannot just approach its minimum, as $g(x)$ would need to have a minimum somewhere, and from there, increase again to be under the polynomial. The function is continuous on a bounded set, hence it must attain its minimum.

More formally, let's assume a strongly convex function over $\mathbb{R}^p$ does not attain it's minimum when approaching it's domain approaches $-\infty$. Then, $\forall x, y \in D_g: y \leq x \Rightarrow f(x) \geq f(y)$. However, from strong convexity, we know that $\forall z \in D_g: f(x) \geq f(z) + \langle \nabla f(x), x - z \rangle + \frac{\mu}{2} \Vert x - z \Vert_2^2$, which is an increasing function as $z \Rightarrow -\infty$ due to $\Vert x - z \Vert_2^2$ growing fastest. But this contradicts the statement that $f$ is decreasing. By contradiction, $f$ must attain its minimum. A similar proof can be constructed for when $x$ approaches $\infty$.

We will now show that $f_\mu$ is smooth, i.e, $\nabla f_\mu$ is L-Lipschitz with respect to the Euclidean norm, with 
\begin{equation}
	L = \|A\|^2_F + \mu \text{, where }\|\cdot\|_F\text{ denotes the Frobenius norm. }
\end{equation}

(1 point for all three questions)

__(i-1)__ Show that $\lambda_{\max}(\mathbf{a}_i\mathbf{a}_i^T) = \left\| \mathbf{a}_i\right\|_2^2$, where $\lambda_{\max}(\cdot)$ denotes the largest eigenvalue.

\begin{equation*}
    \begin{aligned}
        \Vert a_i \Vert_2^2 &= a_i^T a_i\\
        &= Tr(a_i^T a_i)\\
        &= Tr(a_i a_i^T)\\
        &= \sum_{i=1}^{p}\lambda_i(a_i a_i^T) \text{  ($\lambda_i(a_i a_i^T)$ denotes the i-th eigenvalue of $a_i a_i^T$, and is not a product)}
    \end{aligned}
\end{equation*}

Furthermore, we know that

\begin{equation*}
    \begin{aligned}
        a_i a_i^T x &= \lambda x\\
        a_i^T a_i (a_i^T x) &= \lambda (a_i^T x) \text{ (by multiplying both sides with $a_i^T$)} 
    \end{aligned}
\end{equation*}

This means that $a_i^T a_i$ and $a_i a_i^T$ have the same non-zero eigenvalues $\lambda$. However, since $a_i^T a_i$ is one-dimensional, it only has one eigenvalue (equal to itself). Hence, $a_i a_i^T$ only has one non-zero eigenvalue (which is $\geq 0$ since $a_i^T a_i \geq 0$), meaning $\sum_{i=1}^{p}\lambda_i(a_i a_i^T) = \lambda_{max}(a_ia_i^T)$. We can thus conclude that $\Vert a_i \Vert_2^2 = \lambda_{max}(a_ia_i^T)$.

__(i-2)__ Using [2](#mjx-eqn-eq2), show that $\lambda_{\max}(\nabla^2 f_\mu(\mathbf{x})) \leq \sum_{i=1}^{n} \|\mathbf{a}_i\|_2^2 + \mu$. 

__Hint__: Recall that $\lambda_{\max}(\cdot)$ verifies the triangle inequality.

\begin{equation*}
    \begin{aligned}
    \lambda_{\max}(\nabla^2 f_\mu(\mathbf{x})) &\leq \lambda_{\max}(\sum_{i=1}^n \sigma(a_i^Tx)\sigma(-a_i^Tx)a_ia_i^T) + \lambda_{\max}(\mu I)\\
    &\leq \sum_{i=1}^n \lambda_{\max}(\sigma(a_i^Tx)\sigma(-a_i^Tx)a_ia_i^T) + \mu\\
    \end{aligned}
\end{equation*}

Let $c \in \mathbb{R}^p$ and $Ax = \lambda x$. Then, $(cA)x = (c\lambda) x$. Hence a matrix scaled with a scalar $c$ will have its eigenvalues scaled by that constant. Then,

\begin{equation*}
    \begin{aligned}
    \lambda_{\max}(\nabla^2 f_\mu(\mathbf{x})) &\leq \sum_{i=1}^n \sigma(a_i^Tx)\sigma(-a_i^Tx)\lambda_{\max}(a_ia_i^T) + \mu\\
    &= \sum_{i=1}^n \sigma(a_i^Tx)\sigma(-a_i^Tx) \Vert a_i \Vert_2^2 + \mu\\
    \end{aligned}
\end{equation*}

We know that $Range(\sigma(a_i^Tx)) = Range(\sigma(-a_i^Tx)) = (0, 1)$. Hence, $\sigma(a_i^Tx)\sigma(-a_i^Tx) \leq 1$. Substituting that in the equation above gives

\begin{equation*}
    \begin{aligned}
    \lambda_{\max}(\nabla^2 f_\mu(\mathbf{x})) &\leq \sum_{i=1}^n \sigma(a_i^Tx)\sigma(-a_i^Tx) \Vert a_i \Vert_2^2 + \mu\\
    &\leq \sum_{i=1}^n \Vert a_i \Vert_2^2 + \mu
    \end{aligned}
\end{equation*}

__(i-3__ Conclude that $f_\mu$ is $L$-smooth for $L = \|A\|_F^2 + \mu$.

Recall that we can show $f_\mu$ is $L$-smooth by showing that $\nabla^2 f_\mu(x) \preccurlyeq LI$. This represents that all eigenvalues of $\nabla^2 f_\mu(x)$ must be less than or equal to $L$. Without loss of generality, this means that $\lambda_{max}(\nabla^2 f_\mu(x)) \leq L$, since then all other eigenvalues will be $\leq L$ too. We just showed that this holds when $L = \sum_{i=1}^n \Vert a_i \Vert_2^2 + \mu = \Vert A \Vert_F^2 + \mu$. Thus, $f_\mu$ is $L$-smooth for $L = \Vert A \Vert_F^2 + \mu$.


From your work in this section, you have shown that the maximum likelihood estimator for logistic regression might not exist, but it can be guaranteed to exist by adding a $\|\cdot\|_2^2$ regularizer. Consequently, the estimator for $\mathbf{x}^\natural$ we will use will be the solution of the smooth strongly convex problem,
\begin{equation}
	\mathbf{x}^\star=\arg\min_{\mathbf{x} \in \mathbb{R}^p} f(\mathbf{x}) + \frac{\mu}{2}\|\mathbf{x}\|_2^2.
\tag{3}\label{eq:log_problem}
\end{equation}

<a name="cite_note-1"></a>1. [^](#cite_ref-1) TA's will give you candy if you provide a complete proof.